# Stellar particles
This tutorial will teach you how to obtain spectral properties of stellar particles (e.g. representing single stellar populations, SSPs,from a hydrodynamic simulation) from BPASS using `hoki`.

# Prerequisites
In addition to `hoki`, you need a local copy of the BPASS v2.2.1 database (or at least the spectral data for the chab300 IMF). These data are too large to be included in this repository.

In [ ]:
# specify the base path to your BPASS database here
base_path = '~/BPASSv2.2.1_release-07-18-Tuatara/'

# Imports/settings

In [ ]:
from hoki import interpolators
from hoki.spec import bin_luminosity
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

%matplotlib inline
plt.style.use('tuto.mplstyle')

# The stellar particles
Here we just create a dataframe of three stellar particles, this would normally come from a star formation simulation or similar.

In [ ]:
star_parts = pd.DataFrame({
    # absolute initial stellar metallicities
    'metallicity': np.array([1e-2, 1e-4, 1e-3]),
    # log of stellar ages in yr
    'log_age': np.array([6.0, 8.3, 10]),
    # stellar masses in 1e6 M_sun
    'mass': np.array([1, 2.5, 3])
})

# Interpolators
Now let's create interpolator objects for the spectra and emissivities/luminosities provided by BPASS. These interpolator objects simply interpolate SSP properties on the BPASS age-metallicity grids.

In [ ]:
data_path = os.path.join(base_path, 'bpass_v2.2.1_imf_chab300')
imf = 'imf_chab300'

spec_interp = interpolators.SpectraInterpolator(data_path, imf, lam_min=2e2, lam_max=2e3)
em_interp = interpolators.EmissivitiesInterpolator(data_path, imf)

# Results
Let's do the interpolation and see what we get.

In [ ]:
# interpolated SEDs and the corresponding wavelengths
wl, spectra = spec_interp(star_parts['metallicity'], star_parts['log_age'], star_parts['mass'])

plt.figure(figsize=(10,5))
for spec in spectra:
    plt.plot(wl, spec, lw=0.5)
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'SED ($L_\odot/\AA$)')
plt.xscale('log')
plt.yscale('log')

In [ ]:
# interpolated emissivities/luminosities
em = em_interp(star_parts['metallicity'], star_parts['log_age'], star_parts['mass'])
# let's add the to our dataframe (there's probably a more efficient way of doing this)
for col, name in zip(em.T, ['Nion', 'L_Halpha', 'L_FUV', 'L_NUV']):
    star_parts[name] = col

In [ ]:
# this is what the dataframe looks like now
star_parts

# Binning the spectra
`hoki` provides functionality to bin the interpolated spectra, conserving luminosity. This is useful, e.g., when you want to perform a radiative transfer simulation and need to reduce the number of spectral sampling points from what is provided by BPASS.

In [ ]:
bins = np.geomspace(2.1e2, 1.9e3, num=20)
wl_binned, spectra_binned = bin_luminosity(wl, spectra, bins=bins)

plt.figure(figsize=(10,5))
for spec in spectra:
    plt.plot(wl, spec, lw=0.5)
for spec in spectra_binned:
    plt.stairs(spec, edges=bins)
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'SED ($L_\odot/\AA$)')
plt.xscale('log')
plt.yscale('log')